# Sessie 7 - Interpolatie

Voer de onderstaande code uit.

In [ ]:
%matplotlib inline

import csv
import math
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
def list2column(v):
    """Transponeer een lijst van elementen naar een kolomvector."""
    return np.asmatrix(v).transpose()

def column2list(v):
    """Transponeer een kolomvector naar een lijst van elementen."""
    return np.array(v.transpose())[0]

## 1. Algemene lineaire interpolatie

In de cursus hebben we gezien dat we een functie $f(x)$ kunnen benaderen door punten $(x_i, f_i)$ d.m.v. een lineaire interpolatie functie

$$y(x) = \sum\limits_j a_j\Phi_j(x)\ \text{ waarbij }\ y(x_i) = f_i$$

met onbekenden $a_j$ en basisfuncties $\Phi_j(x)$.
Zo geeft $\Phi_j(x) = |x - x_j|$ een stuksgewijze lineaire interpolatie.

Implementeer een methode dat, gegeven een reeks basisfuncties, een interpolatiefunctie berekent voor een functie $f(x)$ door de punten $(x_i, f_i)$.

In [ ]:
def create_linear_bf(x_j):
    """Create a linear basis function."""
    return lambda x: abs(x - x_j)


def interpolate(xis, fis, phis):
    """Returns an interpolation through points (xi, fi) with basis functions phis."""
    # original datapoints (xis)
    # xis = [-1, 0, 2, 5, 8, 10]
    A = np.asmatrix([[phi(xi) for phi in phis] for xi in xis])
    # Turns into matrix (A):
    # [[ 0  1  3  6  9 11]
    #  [ 1  0  2  5  8 10]
    #  [ 3  2  0  3  6  8]
    #  [ 6  5  3  0  3  5]
    #  [ 9  8  6  3  0  2]
    #  [11 10  8  5  2  0]] 
    # Note how A[i][i] = 0, 
    #    as the distance between an element and itself is zero
    
    # Ax = b => x = A^(-1)b
    weights = np.linalg.inv(A)*list2column(fis) 
    weights = column2list(weights)
    def y(x):
        return sum(map(lambda w, phi: w*phi(x), weights, phis))
    return y


In [ ]:
### TEST ###

# Function f
f = lambda x: x*math.sin(x)
xs = np.linspace(-1, 10, 1000)
fs = list(map(f, xs))
plt.title('Function')
plt.plot(xs, fs)

# Data points (xi, fi)
xis = [-1, 0, 2, 5, 8, 10]
fis = list(map(f, xis))
plt.plot(xis, fis, 'bo')

# !! Create linear BFs centered around xi !!
phis = list(map(create_linear_bf, xis))

# Interpolate through (xi, fi) using basis functions phi
y = interpolate(xis, fis, phis)
ys = list(map(y, xs))
plt.plot(xs, ys)
plt.xlim(-1,10)

# Plot basis functions
plt.figure()
plt.title('Basis Functions')
for phi in phis:
    ps = list(map(phi, xs))
    plt.plot(xs, ps)
plt.plot(xis, [0 for _ in range(len(xis))], 'bo')
plt.xlim(-1,10);

In [ ]:
# Plot scaled basis functions
P = np.asmatrix([[phi(xi) for phi in phis] for xi in xis])
weights = np.linalg.solve(P, fis)

plt.figure()
plt.title('Scaled Basis Functions')
for i in range(len(phis)):
    phi = phis[i]
    weight = weights[i]
    ps = list(map(lambda x: weight*phi(x), xs))
    plt.plot(xs, ps)
plt.plot(xis, fis, 'bo')
plt.xlim(-1,10);

## 2. "Lagrangiaanse" lineaire interpolatie

In de les hebben we gezien dat bij polynomiale interpolatie verschillende basissen mogelijk zijn. 

De Lagrangiaanse methode gebruikt als basisfuncties $\Phi_i$:

$$\Phi_i(x_j) =
\left\{ 
  \begin{array}{ c l }
    1 & \quad \textrm{als } i = j \\
    0 & \quad \textrm{als } i \neq j
  \end{array}
\right.$$

Kan je voor stuksgewijs lineaire interpolatie ook dergelijke basisfuncties verzinnen? 
Bereken op basis van deze basisfuncties de interpolant voor het zelfde voorbeeld als voordien. 

In [ ]:
def create_linear_bf_lagrange(z1, z, z2):
    """Create a lagrangian linear basis function.
    
    Arguments 'z1', 'z', 'z2' correspond to three consecutive datapoints:
        z1 = x_{i-1}
        z  = x_{i}
        z2 = x_{i+1}
    """
    # The args are three consecutive datapoints from the dataset.
    # For example: if xis = [0, 1, 5, 6, 7]
    #   One basis function will be centered around xi = 5
    #   Thus, for phi_2(x) corresponds to calling 'create_linear_bf_lagrange(z1=1, z=5, z2=6)'
    def lagrange_basis_function(x):
        if x < z1 or x > z2:
            return 0
        elif x == z:
            return 1
        elif x >= z:
            return (x - z2) / (z - z2)
        else:
            return (x - z1) / (z - z1)
    return lagrange_basis_function

In [ ]:
def create_basis_functions_lagrange(xis):
    """Create the linear approximations with the langrange linear BF using three points."""
    # Problem: what about the first and last element of xis?
    #   Solution: create an extended xis-list with duplicated knots for xis[0] and x[-1]
    #   xis[-1] corresponds to taking the last element of a list
    # For example: if xis = [0, 1, 5, 6, 7]
    #   One basis function will be centered around xi = 7
    #   Thus, phi_4(x) will correspond to calling 'create_linear_bf_lagrange(z1=6, z=7, z2=7)'
    xis_extended = []
    xis_extended.append(xis[0])
    xis_extended.extend(xis)
    xis_extended.append(xis[-1])

    basis_functions = []
    for i in range(0, len(xis_extended) - 2):
        bf = create_linear_bf_lagrange(xis_extended[i],
                                       xis_extended[i+1],
                                       xis_extended[i+2])
        basis_functions.append(bf)    
    return basis_functions

In [ ]:
### TEST ###

# approximate with langrage interpolation
basis_functions_lang = create_basis_functions_lagrange(xis)

# Interpolate through (xi, fi) using basis functions phi.
plt.figure()
plt.title('Function')
plt.plot(xs, fs)
plt.plot(xis, fis, 'bo')

y_p = interpolate(xis, fis, basis_functions_lang)
ys_p = list(map(y_p, xs))
plt.plot(xs, ys_p)
plt.xlim(-1,10)

# Plot basis functions.
plt.figure()
plt.title('Basis Functions')
for phi in basis_functions_lang:
    ps = list(map(phi, xs))
    plt.plot(xs, ps)
plt.plot(xis, [0 for _ in range(len(xis))], 'bo')
plt.xlim(-1,10);

Los de volgende vragen op: 


1. Vergelijk de stuksgewijs lineaire interpolant die je hier verkreeg met degene die je in oefening 1 verkreeg. Wat valt er op? Kan je dit verklaren? <br>
    <b>ANSWER:</b> 
    De twee zijn gelijk. Dat kan ook niet anders want de interpolant is uniek.


2. Hoe ziet de matrix A eruit in het "Lagrangiaans" geval? Kan je dit verklaren? <br>
    <b>ANSWER:</b> 
    We verkrijgen de eenheidsmatrix. Dit komt per constructie uit de eigenschap dat de i'de basisfunctie in alle knopen nul is behalve in i.

## 3. Radial Basis Functions

Bij interpolatie worden Radial Basis Functions (RBF) als een transformatie over de inputwaarden $x_i$, zodat lineaire combinatie tussen deze transformaties voldoende is om ons een goede interpolant te geven.

Deze functies worden vaak gecentreerd rond de geobserveerde punten. Zo is ook $\Phi_j(x) = |x - x_j|$ een soort van RBF die hetzelfde effect geeft als een stuksgewijze lineaire interpolatie.

Een basisfunctie die vaak gebruik wordt is de Gaussiaanse RBF:

$$\Phi(x)_j = \exp\left(-(x-x_j)^2 \right)$$

Deze functie is lineair onafhankelijk wanneer deze gecentreerd wordt op verschillende punten, en geeft een simpele, maar niet-lineaire transformatie van de input-ruimte. Daarnaast geeft deze een interpolant die differentieerbaar is.

Implementeer een functie die deze RBF implementeert en pas de test-code uit de vorige oefening aan zodat deze gebruik maakt van Gaussiaanse RBFs.

In [ ]:
def create_gaussian_rbf(x_j):
    """Create a Gaussian RBF."""
    return lambda x: math.exp(-(x - x_j)**2)

In [ ]:
### TEST ###

# Function f
f = lambda x: x*math.sin(x)
xs = np.linspace(-1, 10, 1000)
fs = list(map(f, xs))
plt.title('Function')
plt.plot(xs, fs)

# Data points (xi, fi)
xis = [-1, 0, 2, 5, 8, 10]
fis = list(map(f, xis))
plt.plot(xis, fis, 'bo')

# Create Gaussian RBFs centered around xi
phis = list(map(create_gaussian_rbf, xis))

# Interpolate through (xi, fi) using basis functions phi
y = interpolate(xis, fis, phis)
ys = list(map(y, xs))
plt.plot(xs, ys)
plt.xlim(-1,10)

# Plot basis functions
plt.figure()
plt.title('Basis Functions')
for phi in phis:
    ps = list(map(phi, xs))
    plt.plot(xs, ps)
plt.plot(xis, [0. for _ in range(len(xis))], 'bo')
plt.xlim(-1,10);

In [ ]:
# Plot scaled basis functions
P = np.asmatrix([[phi(xi) for phi in phis] for xi in xis])
weights = np.linalg.solve(P, fis)

plt.figure()
plt.title('Scaled Basis Functions')
for i in range(len(phis)):
    phi = phis[i]
    weight = weights[i]
    ps = list(map(lambda x: weight*phi(x), xs))
    plt.plot(xs, ps)
plt.plot(xis, fis, 'bo')
plt.xlim(-1,10);

## 4. Omgaan met ruis: interpolatie van globale temperatuur

De volgende datapunten geven de globale temperatuur van 1961 tot 1990 (geschaald tussen 0 en 10):

In [ ]:
fis_temp = []
with open('global_temperature.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    
    next(reader)
    for row in reader:
        fis_temp.append(float(row[1].replace(',', '.')))

fis_temp = fis_temp[::6]
xis_temp = [i*10. / len(fis_temp) for i in range(len(fis_temp))]
plt.title('Global Temperature')
plt.xlabel('Scaled year')
plt.ylabel('Temperature (degrees C)')
plt.plot(xis_temp, fis_temp, 'bo');

Ook al is er een duidelijke, oscillerende trend zichtbaar, het is moeilijk om een goede expliciete functie te bepalen die de observaties verklaart. Daarnaast is er ruis zichtbaar, waardoor de datapunten niet meer overeenkomen met de onderliggende functiewaarden. We zullen nu zien hoe we kunnen omgaan met deze ruis.

In veel applicaties hebben we te maken met data die niet honderd procent correct de onderliggende functie weergeeft (ruis). Als we evenveel basisfuncties gebruiken als datapunten, interpoleren we door deze "foutieve" datapunten, waardoor de functie $f(x)$ niet correct benaderd wordt. Dit fenomeen heet overfitting, aangezien het interpolatiemodel complex genoeg is om deze ruis te "fitten", terwijl de onderliggende functie $f(x)$ meestal simpeler is.

Daarom is het dikwijls beter om het interpolatiemodel simpeler op te stellen (namelijk, met minder basisfuncties), zodat deze noise genegeerd wordt. Dit wil zeggen dat we minder onbekenden (coefficienten van de basisfuncties) hebben dan datapunten, waardoor het stelsel geen oplossing meer biedt die aan alle voorwaarden voldoet (i.e., interpoleer door ieder datapunt).
Met andere woorden, het stelsel $Ax = b \Leftrightarrow x = A^{-1}b$ is niet meer oplosbaar, aangezien $A$ niet vierkant meer is, dus geen inverse $A^{-1}$ heeft.

We kunnen echter proberen een benadering te vinden die "zo goed mogelijk" aan de voorwaarden voldoet.
Het idee is dat we een inverse voor een niet-vierkante matrix zullen definieren. Definieer de inverse voor een niet-vierkante matrix en implementeer een interpolatiemethode voor minder basisfuncties dan datapunten.

(Hint: $Ax = b \Rightarrow A'Ax = A'b$)

<b>ANTWOORD:</b>

We definieren de inverse als volgt:

$$A'Ax = A'b \Leftrightarrow x = (A'A)^{-1}A'b = A^+b$$

$A^+$ noemen we de Moore-Penrose inverse en geeft een Least-Square Error approximatie op de oplossing van het stelsel.
Dit betekent dat we het interpolatiemodel zullen nemen die de som van de 2-normen tussen de geobserveerde en geinterpoleerde punten verkleind.

In [ ]:
def interpolate_LSE(xis, fis, phis):
    """Return an approximate interpolation through points (xi, fi) with basis functions phis."""
    P = np.asmatrix([[phi(xi) for phi in phis] for xi in xis])
    P_inv = np.linalg.inv(P.transpose()*P)*P.transpose()
    weights = P_inv*list2column(fis)
    weights = column2list(weights)
    def y(x):
        return sum(map(lambda w, phi: w*phi(x), weights, phis))
    return y

In [ ]:
### TEST ###

# Plot temperature data
plt.title('Global Temperature')
plt.xlabel('Scaled year')
plt.ylabel('Temperature (degrees C)')
plt.plot(xis_temp, fis_temp, 'bo')

xs = np.linspace(xis_temp[0], xis_temp[-1], 1000)

# Create Gaussian RBFs centered around ci
cis = np.linspace(xis_temp[0], xis_temp[-1], 10)
phis = list(map(create_gaussian_rbf, cis))

# Interpolate through (xi, fi) using basis functions phi
y = interpolate_LSE(xis_temp, fis_temp, phis)
ys = list(map(y, xs))
plt.plot(xs, ys);

In [ ]:
# Plot scaled basis functions
P = np.asmatrix([[phi(xi) for phi in phis] for xi in xis_temp])
P_inv = np.linalg.inv(P.transpose()*P)*P.transpose()
weights = P_inv*list2column(fis_temp)
weights = column2list(weights)

plt.figure()
plt.title('Scaled Basis Functions')
plt.xlabel('Scaled year')
plt.ylabel('Temperature (degrees C)')
for i in range(len(phis)):
    phi = phis[i]
    weight = weights[i]
    ps = list(map(lambda x: weight*phi(x), xs))
    plt.plot(xs, ps)
plt.plot(xis_temp, fis_temp, 'bo');